# JuliaCon 2024
</br>

## End-to-End AI (E2EAI) with Julia, K0s, and Argo Workflow
</br>

#####     Presentor: Paulito Palmes, IBM Research
##### Collaborators: SUNRISE-6G EU Partners
#####          Date: July 11, 2024 
</br>
</br>
</br>
</br>
</br>
</br>
</br>
</br>
</br>
</br>
</br>

<center> 
    <img  src="./eu-funding.png" width="250"/> 
    <img  src="./SUNRISE-6G_QR.png" width="100"/>
    <img  src="./SUNRISE-6G_logo.png" width="100"/>
</center>

# OUTLINE

- ### The Motivations Behind E2EAI (End-to-End AI)
- ### Components of E2EAI
- ### The Julia AI/ML Solution Use-case
- ### The Future

# The Motivations Behind E2EAI

- currently, IaC and MLOPs are treated separately in deploying AI solutions
- issues with no tight integration:
  - difficult to identify optimal infrastructure 
  - difficult to predict resource viability and feasibility
  - difficult to infer the cost of deployment
  - difficult to identify performance bottlenecks and root-cause analysis

# End-to-End AI (E2EAI)
<center>
    <img  src="./yamlcontents.png" width="700"/>
</center>

- E2EAI is a unified framework tightly integrating MLOps and IaC 
  - single yaml file: Infrastructure + ML Pipeline + LifeCycle Management
  - reliance on yaml workflow templates imply zero to minimal coding
  - collection of yamls can be used as inputs to LLM for intent-driven E2EAI

# Components of E2EAI
<center>
    <img  src="./e2eai-components.png" width="700"/>
    <img  src="./eu-funding.png" width="200"/>
</center>

- SUNRISE-6G
  - SUstainable federatioN of Research Infrastructures \
    for Scaling-up Experimentation in 6G
  - H2020 EU Project (3 years)

# The Julia AI/ML Solution Use-case

- AutoMLPipeline workflow
- Integrating AutoMLPipeline in E2EAI

### Load ML pipeline preprocessing components and models

In [11]:
using AutoMLPipeline;
import PythonCall; const PYC=PythonCall; warnings = PYC.pyimport("warnings"); warnings.filterwarnings("ignore")

#### Decomposition
pca = skoperator("PCA"); fa  = skoperator("FactorAnalysis"); ica = skoperator("FastICA")
#### Scaler 
rb   = skoperator("RobustScaler"); pt   = skoperator("PowerTransformer"); norm = skoperator("Normalizer")
mx   = skoperator("MinMaxScaler"); std  = skoperator("StandardScaler")
#### categorical preprocessing
ohe = OneHotEncoder()
#### Column selector
catf = CatFeatureSelector(); numf = NumFeatureSelector(); disc = CatNumDiscriminator()
#### Learners
rf = skoperator("RandomForestClassifier"); gb = skoperator("GradientBoostingClassifier"); lsvc = skoperator("LinearSVC")
svc = skoperator("SVC"); mlp = skoperator("MLPClassifier")
ada = skoperator("AdaBoostClassifier"); sgd = skoperator("SGDClassifier")
skrf_reg = skoperator("RandomForestRegressor"); skgb_reg = skoperator("GradientBoostingRegressor")
jrf = RandomForest(); tree = PrunedTree()
vote = VoteEnsemble(); stack = StackEnsemble(); best = BestLearner();

### Prepare dataset for classification

In [27]:
# Make sure that the input feature is a dataframe and the target output is a 1-D vector.
using AutoMLPipeline
profbdata = getprofb()
X = profbdata[:,2:end] 
Y = profbdata[:,1] |> Vector;
head(x)=first(x,10)
printstyled(head(profbdata))

10×7 DataFrame
 Row │ Home.Away  Favorite_Points  Underdog_Points  Pointspread  Favorite_Name  Underdog_name  Year  
     │ String7    Int64            Int64            Float64      String3        String3        Int64 
─────┼───────────────────────────────────────────────────────────────────────────────────────────────
   1 │ away                    27               24          4.0  BUF            MIA               89
   2 │ at_home                 17               14          3.0  CHI            CIN               89
   3 │ away                    51                0          2.5  CLE            PIT               89
   4 │ at_home                 28                0          5.5  NO             DAL               89
   5 │ at_home                 38                7          5.5  MIN            HOU               89
   6 │ at_home                 34               20          6.0  DEN            KC                89
   7 │ away                    31               21          6.0  LAN     

### Pipeline to transform categorical features to one-hot encoding

In [28]:
pohe = catf |> ohe
tr = fit_transform!(pohe,X,Y)
head(tr)

Row,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,x38,x39,x40,x41,x42,x43,x44,x45,x46,x47,x48,x49,x50,x51,x52,x53,x54,x55,x56
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Pipeline to transform numerical features to pca and ica and concatenate them

In [35]:
pdec = (numf |> pca) + (numf |> ica)
tr = fit_transform!(pdec,X,Y)
head(tr) |> printstyled

10×8 DataFrame
 Row │ x1        x2         x3          x4         x1_1     x2_1       x3_1        x4_1      
     │ Float64   Float64    Float64     Float64    Float64  Float64    Float64     Float64   
─────┼───────────────────────────────────────────────────────────────────────────────────────
   1 │  2.47477    7.87074  -1.10495    -0.902431  1.21219   0.694445  -0.0827759   0.434858
   2 │ -5.47113   -3.82946  -2.08342    -1.00524   1.16818   0.185039   0.559823   -0.855054
   3 │ 30.4068   -10.8073   -6.12339    -0.883938  1.09581   1.28695   -3.01305    -1.89119
   4 │  8.18372  -15.507    -1.43203    -1.08255   1.18184  -0.499186  -0.963083   -1.7005
   5 │ 16.6176    -6.68636  -1.66597    -0.978243  1.19415   0.172818  -1.67191    -0.872769
   6 │ 10.2588     5.22112   0.0731649  -0.928496  1.24383   0.320736  -0.910217    0.453023
   7 │  7.13435    5.60902   0.368661   -0.939797  1.24854   0.227097  -0.601089    0.513793
   8 │ -1.16369   10.3011   -2.15564    -0.86957   1.18

### More complex pipeline with robust scaling and power transform

In [30]:
ppt = (numf |> rb |> ica) + (numf |> pt |> pca)
tr = fit_transform!(ppt,X,Y)
head(tr) |> printstyled

10×8 DataFrame
 Row │ x1          x2         x3         x4       x1_1        x2_1       x3_1         x4_1       
     │ Float64     Float64    Float64    Float64  Float64     Float64    Float64      Float64    
─────┼───────────────────────────────────────────────────────────────────────────────────────────
   1 │  0.0807042  -0.694806  -0.435322  1.21196  -0.64552    -1.40289   -0.0284468    0.111773
   2 │ -0.561567   -0.181312   0.853176  1.1693   -0.832404   -0.475629  -1.14881     -0.01702
   3 │  3.00891    -1.2872     1.89369   1.10255   1.54491    -1.65258   -1.35967     -2.57866
   4 │  0.962134    0.502173   1.69774   1.1853    1.32065    -0.563565  -2.05839     -0.74898
   5 │  1.67015    -0.172989   0.871889  1.19723   1.1223     -1.45555   -0.88864     -0.776195
   6 │  0.908788   -0.322587  -0.454009  1.24404   0.277462   -1.70936    0.00130938   0.0768767
   7 │  0.599852   -0.228538  -0.515151  1.24831   0.0977821  -1.58007   -0.0364638    0.258464
   8 │ -0.33201    -1

### Evaluating complex pipeline with RandomForest learner

In [37]:
prf = (catf |> ohe) + (numf |> rb |> fa) + (numf |> pt |> pca) |> rf
crossvalidate(prf,X,Y,"accuracy_score")

fold: 1, 0.6567164179104478
fold: 2, 0.6417910447761194
fold: 3, 0.7058823529411765
fold: 4, 0.47761194029850745
fold: 5, 0.6865671641791045
fold: 6, 0.7014925373134329
fold: 7, 0.7313432835820896
fold: 8, 0.7352941176470589
fold: 9, 0.7611940298507462
fold: 10, 0.6567164179104478
errors: 0


(mean = 0.6754609306409132, std = 0.07941457013704799, folds = 10, errors = 0)

### Evaluating complex pipeline with Linear SVM learner

In [16]:
plsvc = (catf |> ohe) + (numf |> rb |> fa) + (numf |> pt |> pca) |> lsvc
crossvalidate(plsvc,X,Y,"accuracy_score")

fold: 1, 0.7611940298507462
fold: 2, 0.7910447761194029
fold: 3, 0.6617647058823529
fold: 4, 0.6865671641791045
fold: 5, 0.6268656716417911
fold: 6, 0.7313432835820896
fold: 7, 0.7761194029850746
fold: 8, 0.6911764705882353
fold: 9, 0.7014925373134329
fold: 10, 0.7761194029850746
errors: 0


(mean = 0.7203687445127305, std = 0.0553040309791105, folds = 10, errors = 0)

### Parallel search of the best ML pipeline

In [19]:
using Random, DataFrames, Distributed
nprocs() == 1 && addprocs()
@everywhere using DataFrames; @everywhere using AutoMLPipeline
@everywhere begin
    import PythonCall; const PYC=PythonCall; warnings = PYC.pyimport("warnings"); warnings.filterwarnings("ignore")
end
@everywhere begin
  profbdata = getprofb(); X = profbdata[:,2:end]; Y = profbdata[:,1] |> Vector;
end
@everywhere begin
  jrf  = RandomForest(); ohe  = OneHotEncoder(); catf = CatFeatureSelector(); numf = NumFeatureSelector()
  tree = PrunedTree(); ada  = skoperator("AdaBoostClassifier"); disc = CatNumDiscriminator()
  sgd  = skoperator("SGDClassifier"); std  = skoperator("StandardScaler"); lsvc = skoperator("LinearSVC")
end

learners = @sync @distributed (vcat) for learner in [jrf,ada,sgd,lsvc,tree]
   pcmc = disc |> ((catf |> ohe) + (numf |> std)) |> learner
   println(learner.name[1:end-4])
   mean,sd,_ = crossvalidate(pcmc,X,Y,"accuracy_score",3)
   DataFrame(name=learner.name[1:end-4],mean=mean,sd=sd)
end;

      From worker 3:	AdaBoostClassifier
      From worker 2:	rf
      From worker 5:	LinearSVC
      From worker 6:	prunetree
      From worker 4:	SGDClassifier
      From worker 6:	fold: 1, 0.6473214285714286
      From worker 2:	fold: 1, 0.7142857142857143
      From worker 4:	fold: 1, 0.6964285714285714
      From worker 5:	fold: 1, 0.7142857142857143
      From worker 6:	fold: 2, 0.5758928571428571
      From worker 2:	fold: 2, 0.6339285714285714
      From worker 3:	fold: 1, 0.6473214285714286
      From worker 5:	fold: 2, 0.7098214285714286
      From worker 4:	fold: 2, 0.6383928571428571
      From worker 6:	fold: 3, 0.5714285714285714
      From worker 6:	errors: 0
      From worker 2:	fold: 3, 0.6294642857142857
      From worker 2:	errors: 0
      From worker 5:	fold: 3, 0.7678571428571429
      From worker 5:	errors: 0
      From worker 4:	fold: 3, 0.7276785714285714
      From worker 4:	errors: 0
      From worker 3:	fold: 2, 0.6875
      From worker 3:	fold: 3, 0.651785714

### Best Pipeline

In [10]:
@show sort!(learners,:mean,rev=true);

sort!(learners, :mean, rev = true) = 5×3 DataFrame
 Row │ name                mean      sd
     │ String              Float64   Float64
─────┼─────────────────────────────────────────
   1 │ SGDClassifier       0.729167  0.0220218
   2 │ LinearSVC           0.71875   0.0348672
   3 │ AdaBoostClassifier  0.68006   0.0185863
   4 │ rf                  0.672619  0.0156781
   5 │ prunetree           0.605655  0.0272772


# E2EAI Application

## Infrastructure Creation Automation

<center>
    <img src="./inventory.png">
</center>

## E2EAI Cluster Observability
<center>
     <img src="./dashboard.png"/>
</center>

## AI as a Service: Zero Coding Using Workflow Template
<center>
    <img src="./template.png" width="700"/>
    <img src="./cluster-template.png" width="700"/>
    <img src="./argo-workflow.png" width="700 "/>
</center>

## Explicit ML Pipeline

<center><img src="./mlpipeline.png" width="600"/></center>

## Optimal Pipeline Discovery by AutoML
<center><img src="./lowcode.png" width="500"/></center>

## Low vs High Pipeline Complexity
<center>
    <img src="./argo-submit.png"/>
</center>

### Low Complexity Pipeline for Classification
<center><img src="./low-comp.png" width="500"/></center>

### High Complexity Pipeline for Classification
<center><img src="./high-comp.png" width="500"/></center>

### Sample AutoML Workflow Template
<center><img src="./yaml-template.png"/></center>

# The Future

<center> 
    <img  src="./e2eai-sphere.png" width="800"/> 
    <img  src="./flowchart.png" width="800"/> 
</center>


- ### Unified Control Plane and Intent-Driven E2EAI

# Acknowledgement

<center> 
    <img  src="./eu-funding.png" width="250"/> 
    <img  src="./SUNRISE-6G_QR.png" width="100"/>
    <img  src="./SUNRISE-6G_logo.png" width="100"/>
</center>


#### This work has been funded by the SUNRISE-6G project, Grant number 101139257, co-funded by the European Union and Smart Networks and Services Joint Undertaking (SNS JU).

### DISCLAIMER

##### "Funded by the European Union. Views and opinions expressed are however those of the author(s) only and do not necessarily reflect those of the European Union and Smart Networks and Services Joint Undertaking (SNS JU). Neither the European Union nor the granting authority can be held responsible for them."

<br>
<br>
<br>


<center>
     <font size="+5">THANK YOU!</font>
</center>